In [1]:
import pandas as pd
from datetime import datetime
import re

import os
from docx import Document
from docx2pdf import convert

import warnings
warnings.filterwarnings('ignore')

In [2]:
inv_info = pd.read_excel("inv_info_sample.xlsx")
inv_no_list = inv_info['Invoice No.'].unique()

In [75]:
def get_inv_info(inv_no, sales_tax = 0.1):
    
    use_info = inv_info[inv_info['Invoice No.'] == inv_no].reset_index(drop = True)

    item_dict = {}
    item_dict['CUSTOMER'] = use_info['Customer'].unique()[0]
    item_dict['CUSTOMER_ADDRESS1'] = use_info['Customer Address1'].unique()[0]
    item_dict['CUSTOMER_ADDRESS2'] = use_info['Customer Address2'].unique()[0]
    item_dict['INV_NO'] = use_info['Invoice No.'].unique()[0]
    item_dict['PAYMENT_TERMS'] = use_info['Payment Terms'].unique()[0]
    item_dict['DOC_DATE'] = str(pd.to_datetime(use_info['Invoice Date'].unique()[0]).date())
    item_dict['DUE_DATE'] = str(pd.to_datetime(use_info['Invoice Date'].unique()[0]).date() + pd.Timedelta(days=inv_info['Payment Terms'].unique()[0]))
    item_dict['SUB_AMOUNT'] = 0
    
    # products information
    for i in range(1, 16):
        try:
            item_dict['ITEM' + str(i)] = use_info.loc[i-1, 'Item']
            item_dict['DETAIL' + str(i)] = use_info.loc[i-1, 'Detail']
            item_dict['UNITPRICE' + str(i)] = use_info.loc[i-1, 'Unit Price']
            item_dict['QUAN' + str(i)] = use_info.loc[i-1, 'Quantity']
            item_dict['AMT' + str(i)] = use_info.loc[i-1, 'Unit Price'] * use_info.loc[i-1, 'Quantity']
            
            item_dict['SUB_AMOUNT'] += item_dict['AMT' + str(i)]
            
        except:
            item_dict['ITEM' + str(i)] = ""
            item_dict['DETAIL' + str(i)] = ""
            item_dict['UNITPRICE' + str(i)] = ""
            item_dict['QUAN' + str(i)] = ""
            item_dict['AMT' + str(i)] = ""
    
    item_dict['TAX_AMOUNT'] = round(item_dict['SUB_AMOUNT'] * sales_tax, 2)
    item_dict['FINAL_AMOUNT'] = item_dict['SUB_AMOUNT'] + item_dict['TAX_AMOUNT']
    
    for k in item_dict.keys():
        
        if any(x in k for x in ('UNITPRICE', 'AMT', 'AMOUNT')):
            try:
                item_dict[k] = '{0:,.2f}'.format(item_dict[k])
            except:
                pass
    return item_dict

{'CUSTOMER': 'Customer C',
 'CUSTOMER_ADDRESS1': 'Apt. C, 789 St.',
 'CUSTOMER_ADDRESS2': 'City_C, State_III 30003',
 'INV_NO': 'SAMCO_3',
 'PAYMENT_TERMS': 60,
 'DOC_DATE': '2025-02-16',
 'DUE_DATE': '2025-03-18',
 'SUB_AMOUNT': '45,931.00',
 'ITEM1': 'Product 2',
 'DETAIL1': 'Product 2 details',
 'UNITPRICE1': '59.00',
 'QUAN1': 598,
 'AMT1': '35,282.00',
 'ITEM2': 'Product 10',
 'DETAIL2': 'Product 10 details',
 'UNITPRICE2': '23.00',
 'QUAN2': 463,
 'AMT2': '10,649.00',
 'ITEM3': '',
 'DETAIL3': '',
 'UNITPRICE3': '',
 'QUAN3': '',
 'AMT3': '',
 'ITEM4': '',
 'DETAIL4': '',
 'UNITPRICE4': '',
 'QUAN4': '',
 'AMT4': '',
 'ITEM5': '',
 'DETAIL5': '',
 'UNITPRICE5': '',
 'QUAN5': '',
 'AMT5': '',
 'ITEM6': '',
 'DETAIL6': '',
 'UNITPRICE6': '',
 'QUAN6': '',
 'AMT6': '',
 'ITEM7': '',
 'DETAIL7': '',
 'UNITPRICE7': '',
 'QUAN7': '',
 'AMT7': '',
 'ITEM8': '',
 'DETAIL8': '',
 'UNITPRICE8': '',
 'QUAN8': '',
 'AMT8': '',
 'ITEM9': '',
 'DETAIL9': '',
 'UNITPRICE9': '',
 'QUAN9': '',
 '

In [156]:
# def replace_text_in_doc_table(item_dict, inv_no):
#     doc = Document('inv_template.docx')

#     for table in doc.tables:
#         for row in table.rows:
#             for cell in row.cells:
#                 for para in cell.paragraphs:
#                     # 合并所有 run 的文本
#                     full_text = ''.join([run.text for run in para.runs])

#                     # 替换文本
#                     for i in range(1, 16):
#                         for name in ['ITEM', 'DETAIL', 'UNITPRICE', '#', '@']:
#                             full_text = re.sub(r'\b' + re.escape(name + str(i)) + r'\b',
#                                                str(item_dict.get(name + str(i), '')), full_text)

#                     para.clear()
#                     para.add_run(full_text)

#     new_doc_path = inv_no + '.docx'
#     doc.save(new_doc_path)

# # 调用函数
# replace_text_in_doc_table(item_dict, inv_no)

In [ ]:
# def replace_text_in_doc_table(item_dict, inv_no):
#     doc = Document('inv_template.docx')
    
#     inv_no = item_dict['INV_NO']
#     for table in doc.tables:
#         for row in table.rows:
#             for cell in row.cells:
#                 for para in cell.paragraphs:
#                     for run in para.runs:
#                         for i in range(1, 16):
#                             for name in ['ITEM', 'DETAIL', 'UNITPRICE', 'QUAN', 'AMT']:
#                                  run.text = re.sub(r'\b' + name + str(i) + r'\b', 
#                                                    str(item_dict[name + str(i)]), 
#                                                    run.text)

#     new_doc_path = inv_no + '.docx'
#     doc.save(new_doc_path)
    
# replace_text_in_doc_table(item_dict, inv_no)

In [76]:
def replace_text_in_doc_table(item_dict):
    doc = Document('inv_template.docx')
    
    inv_no = item_dict['INV_NO']
    for table in doc.tables:
        for row in table.rows:
            for cell in row.cells:
                for para in cell.paragraphs:
                    for run in para.runs:
                            for name in item_dict.keys():
                                 run.text = re.sub(r'\b' + name + r'\b', 
                                                   str(item_dict[name]), 
                                                   run.text)

    new_doc_path = inv_no + '.docx'
    doc.save(new_doc_path)
    
replace_text_in_doc_table(item_dict)

In [83]:
for i in inv_info['Invoice No.'].unique():

    item_dict = get_inv_info(i)
    replace_text_in_doc_table(item_dict)
    convert(i + ".docx")
    os.remove(i + ".docx")
    print(i + " generated successfully!")

  0%|          | 0/1 [00:00<?, ?it/s]

SAMCO_1 generated successfully!


  0%|          | 0/1 [00:00<?, ?it/s]

SAMCO_2 generated successfully!


  0%|          | 0/1 [00:00<?, ?it/s]

SAMCO_3 generated successfully!


  0%|          | 0/1 [00:00<?, ?it/s]

SAMCO_4 generated successfully!
